### TAO remote client - License plate character recognition

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)


### The workflow in a nutshell

- Creating a dataset
- Upload OpenALPR dataset to the service
- Getting a PTM from NGC
- Model Actions
    - Train
    - Evaluate
    - Export
    - Convert
    - Inference on TAO
    - Inference on TRT

### Table of contents

1. [Install TAO remote client ](#head-1)
1. [Set the remote service base URL](#head-2)
1. [Access the shared volume](#head-3)
1. [Create the datasets](#head-4)
1. [List datasets](#head-5)
1. [Provide and customize dataset convert specs](#head-6)
1. [Create a model experiment](#head-8)
1. [Find lprnet pretrained model](#head-9)
1. [Customize model metadata](#head-10)
1. [Provide train specs](#head-11)
1. [Run train](#head-12)
1. [Provide evaluate specs](#head-13)
1. [Run evaluate](#head-14)
1. [Provide FP32 export specs](#head-19)
1. [Run FP32 export](#head-20)
1. [Provide model convert specs](#head-23)
1. [Run model convert](#head-24)
1. [Provide TAO inference specs](#head-25)
1. [Run TAO inference](#head-26)
1. [Provide TRT inference specs](#head-27)
1. [Run TRT inference](#head-28)
1. [Delete experiment](#head-30)
1. [Delete datasets](#head-31)
1. [Unmount shared volume](#head-32)
1. [Uninstall TAO Remote Client](#head-33)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import os
import glob
import subprocess
import getpass
import uuid
import json

In [ ]:
namespace = 'default'

### Install TAO remote client <a class="anchor" id="head-1"></a>

In [ ]:
# SKIP this step IF you have already installed the TAO-Client wheel.
! pip3 install nvidia-tao-client

In [ ]:
# View the version of the TAO-Client
! tao-client --version

### FIXME

1. Assign the ip_address and port_number in FIXME 1 and FIXME 2 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
2. Set NGC API key in FIXME 3
3. Assign path of data directory in FIXME 4
4. Choose between default or custom dataset in FIXME 5

### Set the remote service base URL <a class="anchor" id="head-2"></a>

In [ ]:
# Define the node_addr and port number
node_addr = "<ip_address>" # FIXME1 example: 10.137.149.22
node_port = "<port_number>" # FIXME2 example: 32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
%env BASE_URL=http://{node_addr}:{node_port}/{namespace}/api/v1

In [ ]:
# FIXME: Set ngc_api_key valiable
ngc_api_key = "<ngc_api_key>" # FIXME3 example: zZYtczM5amdtdDcwNjk0cnA2bGU2bXQ3bnQ6NmQ4NjNhMDItMTdmZS00Y2QxLWI2ZjktNmE5M2YxZTc0OGyM

# Exchange NGC_API_KEY for JWT
identity = json.loads(subprocess.getoutput(f'tao-client login --ngc-api-key {ngc_api_key}'))

%env USER={identity['user_id']}
%env TOKEN={identity['token']}

### Access the shared volume <a class="anchor" id="head-3"></a>

In [ ]:
# Get PVC ID
pvc_id = subprocess.getoutput(f'kubectl get pvc tao-toolkit-api-pvc -n {namespace} -o jsonpath="{{.spec.volumeName}}"')
print(pvc_id)

In [ ]:
# Get NFS server info
provisioner = json.loads(subprocess.getoutput(f'helm get values nfs-subdir-external-provisioner -o json'))
nfs_server = provisioner['nfs']['server']
nfs_path = provisioner['nfs']['path']
print(nfs_server, nfs_path)

In [ ]:
user = getpass.getuser()
home = os.path.expanduser('~')

! echo "Password for {user}"
password = getpass.getpass()

In [ ]:
# Mount shared volume 
! mkdir -p ~/shared

command = "apt-get -y install nfs-common >> /dev/null"
! echo {password} | sudo -S -k {command}

command = f"mount -t nfs {nfs_server}:{nfs_path}/{namespace}-tao-toolkit-api-pvc-{pvc_id} ~/shared"
! echo {password} | sudo -S -k {command} && echo DONE

### Create the datasets <a class="anchor" id="head-4"></a>

We will be using the `OpenALPR benchmark dataset` for the tutorial. The following script will download the dataset automatically and convert it to the format used by TAO.

**If using custom dataset; it should follow this dataset structure**
```
DATA_DIR
├── train
│   ├── characters.txt
│   ├── image
│   │   ├── image_name_1.jpg
│   │   ├── image_name_2.jpg
|   |   ├── ...
│   └── label
│       ├── image_name_1.txt
│       ├── image_name_2.txt
|       ├── ...
└── val
    ├── characters.txt
    ├── image
    │   ├── image_name_11.jpg
    │   ├── image_name_12.jpg
    |   ├── ...
    └── label
        ├── image_name_11.txt
        ├── image_name_12.txt
        ├── ...
```
The file name should be same for image and label folders

In [ ]:
DATA_DIR = "lprnet_data" # FIXME4
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

In [ ]:
dataset_to_be_used = "default" # FIXME5 example: default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset

In [ ]:
if dataset_to_be_used == "default":
    !python3 -m pip install --upgrade pip
    !python3 -m pip install "opencv-python>=3.4.0.12,<=4.5.5.64"
    !bash ../dataset_prepare/lprnet/download_and_prepare_data.sh $DATA_DIR

In [ ]:
country = "us" # us/ch; us for United States, ch for China

In [ ]:
if dataset_to_be_used == "default":
    character_file_link = "https://api.ngc.nvidia.com/v2/models/nvidia/tao/lprnet/versions/trainable_v1.0/files/{}_lp_characters.txt".format(country)
    !wget -q -O $DATA_DIR/train/characters.txt $character_file_link
    !cp $DATA_DIR/train/characters.txt $DATA_DIR/val/characters.txt 

In [ ]:
train_dataset_id = subprocess.getoutput("tao-client lprnet dataset-create --dataset_type character_recognition --dataset_format lprnet")
print(train_dataset_id)

In [ ]:
! rsync -ah --info=progress2 {DATA_DIR}/train/image ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}/
! rsync -ah --info=progress2 {DATA_DIR}/train/label ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}/
! rsync -ah --info=progress2 {DATA_DIR}/train/characters.txt ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}/
! echo DONE

In [ ]:
eval_dataset_id = subprocess.getoutput("tao-client lprnet dataset-create --dataset_type character_recognition --dataset_format lprnet")
print(eval_dataset_id)

In [ ]:
! rsync -ah --info=progress2 {DATA_DIR}/val/image ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}/
! rsync -ah --info=progress2 {DATA_DIR}/val/label ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}/
! rsync -ah --info=progress2 {DATA_DIR}/val/characters.txt ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}/
! echo DONE

In [ ]:
infer_dataset_id = subprocess.getoutput("tao-client lprnet dataset-create --dataset_type character_recognition --dataset_format raw")
print(infer_dataset_id)

In [ ]:
! rsync -ah --info=progress2 {DATA_DIR}/val/image ~/shared/users/{os.environ['USER']}/datasets/{infer_dataset_id}/
! echo DONE

### List datasets <a class="anchor" id="head-5"></a>

In [ ]:
pattern = os.path.join(home, 'shared', 'users', os.environ['USER'], 'datasets', '*', 'metadata.json')

datasets = []
for metadata_path in glob.glob(pattern):
    with open(metadata_path, 'r') as metadata_file:
        datasets.append(json.load(metadata_file))

print(json.dumps(datasets, indent=2))

### Create a model experiment <a class="anchor" id="head-8"></a>

In [ ]:
network_arch = "lprnet"
model_id = subprocess.getoutput(f"tao-client lprnet model-create --network_arch {network_arch} --encryption_key nvidia_tlt ")
print(model_id)

### Find lprnet pretrained model <a class="anchor" id="head-9"></a>

In [ ]:
pattern = os.path.join(home, 'shared', 'users', '*', 'models', '*', 'metadata.json')

ptm_id = None
for metadata_path in glob.glob(pattern):
  with open(metadata_path, 'r') as metadata_file:
    metadata = json.load(metadata_file)
    metadata_architecture = metadata.get("network_arch")
    ptm_country_info = metadata.get("additional_id_info")
    if metadata_architecture == network_arch and ptm_country_info == country:
      ptm_id = metadata["id"]
      break

print(ptm_id)

### Customize model metadata <a class="anchor" id="head-10"></a>

In [ ]:
metadata_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'metadata.json')

with open(metadata_path , "r") as metadata_file:
    metadata = json.load(metadata_file)

metadata["train_datasets"] = [train_dataset_id]
metadata["eval_dataset"] = eval_dataset_id
metadata["inference_dataset"] = infer_dataset_id
metadata["ptm"] = ptm_id

with open(metadata_path, "w") as metadata_file:
    json.dump(metadata, metadata_file, indent=2)

print(json.dumps(metadata, indent=2))

### Provide train specs <a class="anchor" id="head-11"></a>

In [ ]:
# Default train model specs
! tao-client lprnet model-train-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/train.json

In [ ]:
# Customize train model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'train.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["training_config"]["num_epochs"] = 2

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run train <a class="anchor" id="head-12"></a>

In [ ]:
train_job_id = subprocess.getoutput("tao-client lprnet model-train --id " + model_id)
print(train_job_id)

In [ ]:
def my_tail(logs_dir, log_file):
    %env LOG_FILE={logs_dir}/{log_file}
    ! mkdir -p {logs_dir}
    ! [ ! -f "$LOG_FILE" ] && touch $LOG_FILE && chmod 666 $LOG_FILE
    ! tail -f -n +1 $LOG_FILE | while read LINE; do echo "$LINE"; [[ "$LINE" == "EOF" ]] && pkill -P $$ tail; done

# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
logs_dir = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'logs')
log_file = f"{train_job_id}.txt"

my_tail(logs_dir, log_file)

### Provide evaluate specs <a class="anchor" id="head-13"></a>

In [ ]:
# Default evaluate model specs
! tao-client lprnet model-evaluate-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/evaluate.json

In [ ]:
# Customize evaluate model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'evaluate.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

# Change any spec if you wish

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run evaluate <a class="anchor" id="head-14"></a>

In [ ]:
eval_job_id = subprocess.getoutput(f"tao-client lprnet model-evaluate --id {model_id} --job {train_job_id}")
print(eval_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{eval_job_id}.txt"
my_tail(logs_dir, log_file)

### Provide FP32 export specs <a class="anchor" id="head-19"></a>

In [ ]:
# Default export model specs
! tao-client lprnet model-export-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/export.json

In [ ]:
# Customize export model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'export.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["data_type"] = "fp32"

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run FP32 export <a class="anchor" id="head-20"></a>

In [ ]:
fp32_export_job_id = subprocess.getoutput(f"tao-client lprnet model-export --id {model_id} --job {train_job_id}")
print(fp32_export_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{fp32_export_job_id}.txt"
my_tail(logs_dir, log_file)

### Provide model convert specs <a class="anchor" id="head-23"></a>

In [ ]:
# Default convert model specs
! tao-client lprnet model-convert-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/convert.json

In [ ]:
# Customize convert model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'convert.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["t"] = "fp16"
specs["p"] = "image_input,1x3x48x96,4x3x48x96,16x3x48x96"

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run model convert <a class="anchor" id="head-24"></a>

In [ ]:
convert_job_id = subprocess.getoutput(f"tao-client lprnet model-convert --id {model_id} --job {fp32_export_job_id}")
print(convert_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{convert_job_id}.txt"
my_tail(logs_dir, log_file)

### Provide TAO inference specs <a class="anchor" id="head-25"></a>

In [ ]:
# Default inference model specs
! tao-client lprnet model-inference-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/inference.json

In [ ]:
# Customize TAO inference specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'inference.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

# Change any spec if you wish

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run TAO inference <a class="anchor" id="head-26"></a>

In [ ]:
tlt_inference_job_id = subprocess.getoutput(f"tao-client lprnet model-inference --id {model_id} --job {train_job_id}")
print(tlt_inference_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{tlt_inference_job_id}.txt"
my_tail(logs_dir, log_file)

In [ ]:
job_dir = f"{home}/shared/users/{os.environ['USER']}/models/{model_id}/{tlt_inference_job_id}"
# You can view the predictions here
!cat {job_dir}/logs_from_toolkit.txt

### Provide TRT inference specs <a class="anchor" id="head-27"></a>

In [ ]:
# Default inference model specs
! tao-client lprnet model-inference-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/inference.json

In [ ]:
# Customize TAO inference specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'inference.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

# Change any spec if you wish

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run TRT inference <a class="anchor" id="head-28"></a>

In [ ]:
trt_inference_job_id = subprocess.getoutput(f"tao-client lprnet model-inference --id {model_id} --job {fp32_export_job_id}")
print(trt_inference_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{trt_inference_job_id}.txt"
my_tail(logs_dir, log_file)

In [ ]:
job_dir = f"{home}/shared/users/{os.environ['USER']}/models/{model_id}/{tlt_inference_job_id}"
# You can view the predictions here
!cat {job_dir}/logs_from_toolkit.txt

### Delete experiment <a class="anchor" id="head-30"></a>

In [ ]:
! rm -rf ~/shared/users/{os.environ['USER']}/models/{model_id}
! echo DONE

### Delete datasets <a class="anchor" id="head-31"></a>

In [ ]:
! rm -rf ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}
! rm -rf ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}
! rm -rf ~/shared/users/{os.environ['USER']}/datasets/{infer_dataset_id}
! echo DONE

### Unmount shared volume <a class="anchor" id="head-32"></a>

In [ ]:
command = "umount ~/shared"
! echo {password} | sudo -S -k {command} && echo DONE

### Uninstall TAO Remote Client <a class="anchor" id="head-33"></a>

In [ ]:
! pip3 uninstall -y nvidia-tao-client